In [ ]:
%run D:/UTD/UTDFall2023/Calibration-of-LoRaNodes-using-Super-Learners/plots/Scatter_Plot.ipynb
%run D:/UTD/UTDFall2023/Calibration-of-LoRaNodes-using-Super-Learners/plots/QQ_Plot.ipynb
# %run D:/UTD/UTDFall2023/Calibration-of-LoRaNodes-using-Super-Learners/plots/Feature_Importance_Plot_Random_Forest_Regressor.ipynb

In [ ]:
def Stacking_Regression(X_train, X_test, y_train, y_test, filtered_data):
    ml_type = 'SL'

    # Define base learners
    rf_regressor = RandomForestRegressor(n_estimators = 50, random_state = 0)

    bagging_regressor = BaggingRegressor(estimator = DecisionTreeRegressor(), n_estimators=10, random_state=42)

    lgbm_regressor = LGBMRegressor(n_estimators=100, max_depth=-1, learning_rate=0.1, num_leaves=31, min_child_samples=20, random_state=1)

    dt_regressor = DecisionTreeRegressor(hidden_layer_sizes=(64, 64), activation='relu', random_state=1, max_iter=500)

    def rf_meta_learner_cv(n_estimators, max_depth, min_samples_leaf, min_samples_split):
        n_estimators = int(n_estimators)
        max_depth = int(max_depth)
        min_samples_leaf = int(min_samples_leaf)
        min_samples_split = int(min_samples_split)

        model = StackingRegressor(
            estimators=[
                ('rf', rf_regressor),
                ('bagging', bagging_regressor),
                ('lgbm', lgbm_regressor),
                ('nn', nn_regressor)
            ],
            final_estimator=RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth,
                                                  min_samples_leaf=min_samples_leaf, 
                                                  min_samples_split=min_samples_split, 
                                                  random_state=0)
        )

        model.fit(X_train, y_train)
        predict_test = model.predict(X_test)
        return r2_score(y_test, predict_test)

    # Bayesian Optimization
    pbounds = {
        'n_estimators': (1, 200),
        'max_depth': (1, 15),
        'min_samples_leaf': (1, 10),
        'min_samples_split': (2, 10)
    }

    optimizer = BayesianOptimization(
        f=rf_meta_learner_cv,
        pbounds=pbounds,
        random_state=1,
    )
    optimizer.maximize(init_points=10, n_iter=25)

    # Optimal parameters for RandomForest Meta Learner
    optimal_params = optimizer.max['params']
    optimal_n_estimators = int(optimal_params['n_estimators'])
    optimal_max_depth = int(optimal_params['max_depth'])
    optimal_min_samples_leaf = int(optimal_params['min_samples_leaf'])
    optimal_min_samples_split = int(optimal_params['min_samples_split'])
    print("Optimal Parameters for RandomForest Meta Learner:", optimal_params)

    # Final model with tuned RandomForest Meta Learner
    final_model = StackingRegressor(
        estimators=[
            ('rf', rf_regressor),
            ('bagging', bagging_regressor),
            ('lgbm', lgbm_regressor),
            ('nn', nn_regressor)
        ],
        final_estimator=RandomForestRegressor(n_estimators=optimal_n_estimators, max_depth=optimal_max_depth,
                                              min_samples_leaf=optimal_min_samples_leaf, 
                                              min_samples_split=optimal_min_samples_split, 
                                              random_state=0)
    )
    final_model.fit(X_train, y_train)

    # Predictions and Evaluation
    predict_train = final_model.predict(X_train)
    predict_test = final_model.predict(X_test)

    train_df = pd.DataFrame({'Actual': y_train, 'Predicted': predict_train, 'Category': 'Training'}, index=y_train.index)
    test_df = pd.DataFrame({'Actual': y_test, 'Predicted': predict_test, 'Category': 'Testing'}, index=y_test.index)

    combined_df = pd.concat([train_df, test_df])
    combined_df = combined_df.sort_index()

    r2_score_train = round(r2_score(y_train, predict_train), 2)
    r2_score_test = round(r2_score(y_test, predict_test), 2)

    print('R2 Score Train:', r2_score_train)
    print('R2 Score Test:', r2_score_test)

    # Assuming you have some function to plot results
    # Scatter_Plot(combined_df, train_df, test_df, r2_score_train, r2_score_test, v, unit, ml_type)
    # qq_plot(test_df, v, unit, ml_type)

    return r2_score_test